In [ ]:
"""
This notebook is obselete and has been broken into:
vt_contra_metadata_filtering
vt_contra_text_preprocessing
...

"""

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import glob
import json
import re
import pprint

In [2]:
pq_files = glob.glob('../resources/v8_preprocessed/*parquet')

metadata_df = pd.read_csv('../resources/metadata.csv')

pq_vector_files = glob.glob('../resources/v8_vectors/*')
pq_text_files = glob.glob('../resources/v8_preprocessed/*')
paired_text_vector_files = list(zip(sorted(pq_text_files), sorted(pq_vector_files)))

In [12]:
"""
def extract_sections_and_vectors(text_df, vector_df, uid_list, sections):
    
    text_df_with_ids = text_df.loc[text_df.cord_uid.isin(uid_list)]
    text_df_of_sections = text_df_with_ids.loc[text_df_with_ids.section.isin(sections)]
    sentence_dict = text_df_of_sections.groupby('cord_uid')['sentence'].apply(list).to_dict()
    
    vector_df_with_ids = vector_df.loc[vector_df.cord_uid.isin(uid_list)]
    vector_df_of_sections = vector_df_with_ids.loc[vector_df_with_ids.section.isin(sections)]
    vector_dict = text_df_of_sections.groupby('cord_uid')['sentence'].apply(list).to_dict()
"""

def extract_sections(text_df, uid_list, sections):
    """
    Returns a dict of {paper_id: section: sentences} for specified paper id and specified sections
    """
    
    text_df_with_ids = text_df.loc[text_df.cord_uid.isin(uid_list)]
    text_df_of_sections = text_df_with_ids.loc[text_df_with_ids.section.isin(sections)]
    section_dict = text_df_of_sections.groupby('cord_uid')['sentence'].apply(list).to_dict()

    #concatenate sentences
    section_concat_sentences_dict = {k: " ".join(v) for k,v in section_dict.items()}
    
    return section_concat_sentences_dict

def extract_sections_from_pq_files_by_uid(pq_files, uid_list, sections_oi):
    """
    Iterate through v8 preprocessed parquet files, extracting papers by cord_uid and specified sections.
    """
    master_dict = {}
      
    for pq_file in pq_files:
        print("Processing %s..." % pq_file)
        text_df = pd.read_parquet(pq_file)
        text_df.loc[:, 'sentence'] = text_df.loc[:, 'sentence'].astype(str)
        text_df.loc[:, 'section'] = text_df.loc[:, 'section'].astype(str)

        for section in sections_oi:
            print("Extracting %s section..." % section)
            
            if section not in master_dict.keys():
                master_dict[section] = {}
                
            tmp_dict = extract_sections(text_df, uid_list, [section])
            for k, v in tmp_dict.items():
                master_dict[section].setdefault(k, []).append(v)
        
    return master_dict

def extract_paper_section_headers(pq_files, cord_uids=[]):
    """
    Extract all section headers from parquet text files with specified cord uids.
    
    Return as dictionary {cord_uid : [section headers]}
    """
    cord_uid_section_header_dict = {}
    for pq_file in pq_files:
        print("Processing %s..." % pq_file)
        text_df = pd.read_parquet(pq_file)
        
        #Filter to cord_uids if specified, else include all
        if cord_uids:
            text_df = text_df.loc[text_df.cord_uid.isin(cord_uids)]
        
        section_dict = text_df.groupby('cord_uid')['section'].apply(list).to_dict()
        section_dict_unique = {k: list(set(v)) for k,v in section_dict.items()}
        
        for cord_uid, sections in section_dict_unique.items():
            for section in sections:
                cord_uid_section_header_dict.setdefault(cord_uid, set([])).add(section)
    
    return cord_uid_section_header_dict

def extract_paper_sections(pq_files, cord_uids=[]):
    """
    Extract all section texts from parquet text files with specified cord uids.
    
    Return as dictionary {cord_uid : [section texts]}
    """
    cord_uid_section_text_dict = {}
    for pq_file in pq_files:
        print("Processing %s..." % pq_file)
        text_df = pd.read_parquet(pq_file)
        
        if cord_uids:
            text_df = text_df.loc[text_df.cord_uid.isin(cord_uids)]
        
        uid_text_dict = dict(tuple(text_df.groupby('cord_uid')))
        for cord_uid, paper_df in uid_text_dict.items():
            if cord_uid not in cord_uid_section_text_dict:
                master_dict[cord_uid] = {}
                
            section_sentences_dict = dict(tuple(paper_df[['section', 'sentence']].groupby('section')))
            for section, sentence_df in section_sentences_dict.items():
                sentence_df.loc[:, 'sentence'] = sentence_df.sentence.fillna('')
                sentences = sentence_df.sentence.tolist()
                
                current_sentences = cord_uid_section_text_dict[cord_uid].get(section, '')
                new_sentences = current_sentences + ' ' + ' '.join(sentences)
                cord_uid_section_text_dict[cord_uid][section] = new_sentences
                
    return cord_uid_section_text_dict

In [3]:
#Load metadata and generate a filtered list of cord_uids for papers containing covid-19 terms in title or abstract

metadata_df.loc[:, 'title_abstract'] = metadata_df.loc[:, 'title'].str.lower() + ' ' + metadata_df.loc[:, 'abstract'].str.lower()
metadata_df.loc[:, 'title_abstract'] = metadata_df.loc[:, 'title_abstract'].fillna('')

#Prior covid19 term list
#covid19_df = metadata_df.loc[metadata_df.title_abstract.str.contains('covid-19|sars-cov-2|2019-ncov|sars coronavirus 2|2019 novel coronavirus')]

with open('../resources/covid_19_terms_200427.txt') as f:
    covid_19_terms = f.read().splitlines()
    covid_19_term_pattern = '|'.join([i.lower() for i in covid_19_terms])

covid19_df = metadata_df.loc[metadata_df.title_abstract.str.contains(covid_19_term_pattern)]
print("Covid-19 df length: %d" % len(covid19_df))

covid19_uid_list = covid19_df.cord_uid.tolist()

Covid-19 df length: 4424


In [10]:
"""
Covid-19 was identified in December 2019.  Papers published before this point are unlikely to be relevant.

Filtering to papers published after 10-01-2019 to allow for papers before official identification.
"""

covid19_date_filtered_df = covid19_df.loc[covid19_df['publish_time'] > '2019-10-01']
print("Covid-19 df filtered to publication dates after 2019-10-01: %d" % len(covid19_date_filtered_df))

date_incl_cord_uids = set(covid19_date_filtered_df.cord_uid.tolist())
date_excl_uids = set(covid19_uid_list) - set(date_incl_cord_uids)

print("Papers filtered out by date cutoff:")
pprint.pprint(covid19_df.loc[covid19_df.cord_uid.isin(date_excl_uids)].head())

Covid-19 df filtered to publication dates after 2019-10-01: 4150
Papers filtered out by date cutoff:
     cord_uid                                                sha source_x  \
12   qj4dh6rg           3ed670f60a7be2e3e2a991ea8af1fdd5fa5e2b2c      PMC   
29   fpj5urao  e9c78584c08ba79d735e150eff98297eb57f12dd; cdb2...      PMC   
53   ic4d9dhk           72ace5af731fdf4c384e912b074193d13902b7a1      PMC   
54   jh9e85c0           d16e1bd4c6cae1bb55dbc695c5e2ba5105a13921      PMC   
103  eqiokrub           10e691d6bf9cfc219bb72a2761d37ae5f1677f77      PMC   

                                                 title  \
12   Cloaked similarity between HIV-1 and SARS-CoV ...   
29   Moderate mutation rate in the SARS coronavirus...   
53   Expression profile of immune response genes in...   
54   Molecular mechanisms of severe acute respirato...   
103  Croup Is Associated with the Novel Coronavirus...   

                              doi       pmcid   pubmed_id license  \
12         10.1186

In [13]:
uid_section_headers_dict = extract_paper_section_headers(pq_files, date_incl_cord_uids)

Processing resources/v8_preprocessed/v8processedText1.parquet...
Processing resources/v8_preprocessed/v8processedText4.parquet...
Processing resources/v8_preprocessed/v8processedText8.parquet...
Processing resources/v8_preprocessed/v8processedText10.parquet...
Processing resources/v8_preprocessed/v8processedText3.parquet...


KeyboardInterrupt: 

In [147]:
from collections import defaultdict
import re


"""
Identify the list of UIDs containing each unique section header and also count them.
"""

section_uids_dict = defaultdict(set)
section_counts_dict = defaultdict(int)

for uid, sections in uid_section_headers_dict.items():
    for section in sections:
        section_uids_dict[section].add(uid)
        section_counts_dict[section] += 1

In [111]:
with open('covid19_paper_section_header_freq_200427.txt', 'w') as f:
    for i in sorted(section_counts_dict.items(), key=lambda k_v: k_v[1], reverse=True):
        f.write('%s\t%s\n' % (i[0], i[1]))

In [44]:
sect_dict = extract_sections_from_pq_files_by_uid(pq_files, covid19_uid_list, ['title', 'abstract'])

Processing resources/v8_preprocessed/v8processedText1.parquet...
Extracting title section...
Extracting abstract section...
Processing resources/v8_preprocessed/v8processedText4.parquet...
Extracting title section...
Extracting abstract section...
Processing resources/v8_preprocessed/v8processedText8.parquet...
Extracting title section...
Extracting abstract section...
Processing resources/v8_preprocessed/v8processedText10.parquet...
Extracting title section...
Extracting abstract section...
Processing resources/v8_preprocessed/v8processedText3.parquet...
Extracting title section...
Extracting abstract section...
Processing resources/v8_preprocessed/v8processedText0.parquet...
Extracting title section...
Extracting abstract section...
Processing resources/v8_preprocessed/v8processedText12.parquet...
Extracting title section...
Extracting abstract section...
Processing resources/v8_preprocessed/v8processedText15.parquet...
Extracting title section...
Extracting abstract section...
Proce

In [112]:
covid19_df.to_csv('../output/vt_contra_v9_covid19_metadata_200427.csv')

In [71]:
#Generate a mapping of cord_uids to text files to facilitate extraction

all_cord_uids = metadata_df.cord_uid.tolist()

print("Number of cord_uids in current version: %d" % len(all_cord_uids))
print("Number of unique cord_uids in current version: %d" % len(set(all_cord_uids)))

Number of cord_uids in current version: 57366
Number of unique cord_uids in current version: 57332


In [80]:
#Non-unique cord_uids
duplicated_cord_uid_df = metadata_df[metadata_df.duplicated(subset=['cord_uid'], keep=False)]
duplicated_cord_uid_df.sort_values(by='cord_uid')

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,title_abstract
29489,0klupmep,NaN,Elsevier,Infectious disease surveillance update,10.1016/s1473-3099(19)30075-1,NaN,30833065.0,els-covid,NaN,2019-03-31,"Zwizwai, Ruth",The Lancet Infectious Diseases,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/s1473-3099(19)30075-1,
16421,0klupmep,NaN,PMC,Infectious disease surveillance update,10.1016/s1473-3099(19)30075-1,PMC7129894,30833064.0,no-cc,NaN,2019-02-27,"Zwizwai, Ruth",Lancet Infect Dis,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,
16419,0z5wacxs,7e787fd2ae5b544add6281d3d40ad322de26aa17,PMC,Transportation capacity for patients with high...,10.1111/1469-0691.12290,PMC7128608,25636943.0,no-cc,Highly infectious diseases (HIDs) are defined ...,2015-06-22,"Schilling, S.; Maltezou, H.C.; Fusco, F.M.; De...",Clin Microbiol Infect,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,transportation capacity for patients with high...
29293,0z5wacxs,7e787fd2ae5b544add6281d3d40ad322de26aa17,Elsevier,Transportation capacity for patients with high...,10.1111/1469-0691.12290,NaN,24750421.0,els-covid,Abstract Highly infectious diseases (HIDs) are...,2019-04-30,"Schilling, S.; Maltezou, H.C.; Fusco, F.M.; De...",Clinical Microbiology and Infection,NaN,NaN,True,False,custom_license,https://doi.org/10.1111/1469-0691.12290,transportation capacity for patients with high...
28740,21htepa1,a25e212b03cc65c44dcc336775b101934e30f041,Elsevier,Panspermia—true or false?,10.1016/s0140-6736(03)14040-8,NaN,12907025.0,els-covid,NaN,2003-08-02,"de Leon, Samuel Ponce; Lazcano, Antonio",The Lancet,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/s0140-6736(03)14040-8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28771,vp5358rr,f3eefad19d628cbe1fc0d65a6322a60d6cc5a3ab,Elsevier,PARENTAL ORIGIN OF CHROMOSOME 15 DELETION IN P...,10.1016/s0140-6736(83)92745-9,NaN,6134086.0,els-covid,NaN,1983-06-04,"Butler, MerlinG.; Palmer, CatherineG.",The Lancet,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/s0140-6736(83)92745-9,
15934,vqbreyna,2513ddf57215cb9297e05db838156d8856c8d6a5,PMC,"COVID-19, A Clinical Syndrome Manifesting as H...",10.3947/ic.2020.52.1.110,PMC7113449,32153144.0,cc-by-nc,NaN,2020-03-10,"Song, Young Goo; Shin, Hyoung-Shik",Infect Chemother,NaN,NaN,True,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,
41120,vqbreyna,NaN,WHO,"COVID-19, A Clinical Syndrome Manifesting as H...",NaN,NaN,32153144.0,unk,NaN,2020,"Song, Young Goo; Shin, Hyoung Shik",Infect Chemother,NaN,#6103,False,False,NaN,https://www.ncbi.nlm.nih.gov/pubmed/32153144/,
28439,xjpev4jw,c05522c7132d3162d433c02ade1fc80604625a4f,Elsevier,"Virus-Specific Antibody, in the Absence of T C...",10.1016/s0002-9440(10)62301-2,NaN,15743792.0,els-covid,Mice infected with mouse hepatitis virus strai...,2005-03-31,"Kim, Taeg S.; Perlman, Stanley",The American Journal of Pathology,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/s0002-9440(10)62301-2,"virus-specific antibody, in the absence of t c..."


In [31]:
#Functions for mapping cord_uids to text files, to make retrieval of specific papers faster
#Not yet using these

def extract_column_from_pq_file(pq_file, col, unique_flag=False):
    
    print("Extracting %s values from %s..." % (col, pq_file))
    pq_df = pd.read_parquet(pq_file)
    
    if unique_flag:
        res_list = set(pq_df[col].tolist())
    else:
        res_list = pq_df[col].tolist()
    
    print("Number of %s values extracted from %s: %d" % (col, pq_file, len(res_list)))
    
    return res_list
    
def map_pq_files_to_col_vals(pq_files, col, outfile= '', unique_flag=False):
    
    res_dict = {}
    
    for pq_file in pq_files:
        pq_name = pq_file.split('.')[0]
        res_dict[pq_file] = extract_column_from_pq_file(pq_file, col, unique_flag)
    
    if outfile:
        with open(outfile, 'w') as f:
            json.dump(res_dict)
    
    return res_dict

def map_col_vals_to_pq_files(pq_files, col, outfile = '', unique_flag=False):
    
    res_dict = {}
    
    for pq_file in pq_files:
        col_vals_in_pq = extract_column_from_pq_file(pq_file, col, unique_flag)
        for col_val in col_vals_in_pq:
            res_dict.setdefault(col_val, []).append(pq_file)
    
    if outfile:
        print("Writing dict to %s" % outfile)
        with open(outfile, 'w') as f:
            json.dump(res_dict, f)
    
    return res_dict

def generate_file_list_for_extraction(col_val_files_dict, col_vals):
    
    file_list = set([])
    
    for col_val, files in col_val_files_dict.items():
        for file in files:
            file_list.add(file)
            
    return file_list
    

In [30]:
cord_uids_pq_files_dict = map_col_vals_to_pq_files(pq_files, 'cord_uid', outfile='cord_uids_pq_file_map_v8_200425.json', unique_flag=True)

Extracting cord_uid values from resources/v8_preprocessed/v8processedText1.parquet...
Number of cord_uid values extracted from resources/v8_preprocessed/v8processedText1.parquet: 2375
Extracting cord_uid values from resources/v8_preprocessed/v8processedText4.parquet...
Number of cord_uid values extracted from resources/v8_preprocessed/v8processedText4.parquet: 2772
Extracting cord_uid values from resources/v8_preprocessed/v8processedText8.parquet...
Number of cord_uid values extracted from resources/v8_preprocessed/v8processedText8.parquet: 2325
Extracting cord_uid values from resources/v8_preprocessed/v8processedText10.parquet...
Number of cord_uid values extracted from resources/v8_preprocessed/v8processedText10.parquet: 1188
Extracting cord_uid values from resources/v8_preprocessed/v8processedText3.parquet...
Number of cord_uid values extracted from resources/v8_preprocessed/v8processedText3.parquet: 2065
Extracting cord_uid values from resources/v8_preprocessed/v8processedText0.par

In [6]:
"""
Functions to generate regex match patterns from synonymous words/phrases for filtering subject headers
"""

import re

def extract_regex_pattern(section_list, pattern):
    r = re.compile(pattern, re.IGNORECASE)
    extracted_list = list(filter(r.match, section_list))
    remaining_list = list(set(section_list) - set(extracted_list))
    
    return remaining_list, extracted_list

def construct_regex_match_pattern(terms):
    terms = ['.*%s.*' % i for i in terms]
    pattern = '|'.join(terms)
    return pattern

In [9]:
unique_sections = [str(i) for i in section_counts_dict.keys()]

#Examples
#figure and table references

figref_terms = ['figref', 'tabref', 'figure']
figref_pattern = construct_regex_match_pattern(figref_terms)
remaining_list, extracted_list = extract_regex_pattern(unique_sections, figref_pattern)
print("Figure and table reference headers:")
print(extracted_list)


NameError: name 'section_counts_dict' is not defined

In [ ]:
"""
Determine how many unique papers have at least 1 discussion section using one of the discussion/conclusion headers.

conclusion
discussion
interpretation
added value of this study
research in context
concluding
closing remarks
summary of findings
"""

"""
for header, count in section_counts_dict.items():
    if count > 1 and count < 5:
        print(header)
"""

In [7]:
"""
Construct regex pattern for discussion header terms and extract list of matching headers.

"""
discussion_terms = [
    'conclusion',
    'discussion',
    'interpretation',
    'added value of this study',
    'research in context',
    'concluding',
    'closing remarks',
    'summary of findings',
    'outcome'
]

conc_pattern = construct_regex_match_pattern(discussion_terms)
print(conc_pattern)

.*conclusion.*|.*discussion.*|.*interpretation.*|.*added value of this study.*|.*research in context.*|.*concluding.*|.*closing remarks.*|.*summary of findings.*|.*outcome.*


In [8]:
rem_header_list, ext_header_list = extract_regex_pattern(unique_sections, conc_pattern)

with open('discussion_headers_list_200429.txt', 'w') as f:
    
    for ext_header in ext_header_list:
        f.write("%s\t%d\n" %(ext_header, section_counts_dict[ext_header]))

NameError: name 'unique_sections' is not defined

In [298]:
"""
Determine how many uids contain at least 1 of the extracted headers.
"""

covered_uids = set([])

for header in ext_header_list:
    for uid in section_uids_dict[header]:
        covered_uids.add(uid)
    
uncovered_uids = set(covid19_uid_list) - covered_uids

print("UIDs containing at least 1 of the extracted headers: %d" % len(covered_uids))
print("UIDs not containing any of the extracted headers: %d" % len(uncovered_uids))

UIDs containing at least 1 of the extracted headers: 1687
UIDs not containing any of the extracted headers: 2736


In [299]:
"""
Determine the reasons for the 2753 uncovered uids.
"""

uid_sect_header_artifacts = {
    'uids_with_body_text': set([]),
    'uids_with_only_title_and_abstract' : set([]),
    'uids_with_no_text' : set([]),
    'unaccount_uids' : set([])

}

unaccount_uids = set([])
for uid in list(uncovered_uids):
    
    sect_headers = uid_section_headers_dict.get(uid)
    account_flag = 0
    if sect_headers:
        
        if len(set(sect_headers) - set(['abstract', 'title'])) == 0:
            uid_sect_header_artifacts['uids_with_only_title_and_abstract'].add(uid)
            account_flag = 1
        elif 'body_text' in sect_headers:
            uid_sect_header_artifacts['uids_with_body_text'].add(uid)
            account_flag = 1
    else:
        uid_sect_header_artifacts['uids_with_no_text'].add(uid)
        account_flag = 1
    
    if account_flag == 0:
        uid_sect_header_artifacts['unaccount_uids'].add(uid)
    

In [300]:
total_uids = set([])

for art_type, uids in uid_sect_header_artifacts.items():
    print(art_type, len(uids))
    
    for uid in uids:
        total_uids.add(uid)

uids_with_body_text 400
uids_with_only_title_and_abstract 1129
uids_with_no_text 781
unaccount_uids 426


In [301]:
for uid in uid_sect_header_artifacts['uids_with_body_text']:
    print(uid)
    print(uid_section_headers_dict[uid])

j40zj4al
{'body_text', '114', '232', 'title', 'abstract', '(which was not peer-reviewed)', '115', '317'}
1mchg5yg
{'body_text', 'abstract', 'title'}
f87h5qh6
{'body_text', 'abstract', '5', 'title'}
c800ynvc
{'body_text', 'abstract', 'title'}
umqowine
{'body_text', 'title', 'abstract', '11', '(which was not peer-reviewed)', '57.'}
8g70j0qw
{'body_text', 'abstract', 'title'}
z2g1mx1p
{'body_text', 'A Framework for Collecting and Sharing Data', 'title', 'Introduction', 'abstract', 'The Need for Information'}
0xciml6s
{'body_text', 'Additional Resources', 'Environment', 'Imaging Logistics During Pandemic', 'COVID-19 on a CT scan. (Q14)', 'title', 'This', 'abstract', 'CT scan is indicated in a patient who has functional impairment and/or hypoxemia after recovery from COVID-19 (Q13)', 'Resource constraints', 'Scenario 1: Mild Features of COVID-19', 'Scenario 2: Moderate to Severe Features of COVID-19', 'Disease progression', 'The Use of Imaging in COVID-19', 'Daily chest radiographs are not 

In [302]:
"""
As a starting point, convert the covid19_df (metadata filtered to covid-19 papers) into a dict with title and abstract sections.

"""


covid19_df_copy = covid19_df.copy()
covid19_df_copy = covid19_df_copy.set_index('cord_uid')
covid19_title_abstract_df = covid19_df_copy[['title', 'abstract']]
covid19_title_abstract_df.loc[:, 'title'] = covid19_title_abstract_df.loc[:, 'title'].str.lower()
covid19_title_abstract_df.loc[:, 'abstract'] = covid19_title_abstract_df.loc[:, 'abstract'].str.lower()
covid19_title_abstract_dict = covid19_title_abstract_df.transpose().to_dict()

/home/slander/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/home/slander/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  if sys.path[0] == '':


In [303]:
"""
Extract sections from covid-19 papers, using the 242 headers extracted by regex pattern.

"""

sect_dict = extract_sections_from_pq_files_by_uid(pq_files, covid19_uid_list, ext_header_list)

Processing resources/v8_preprocessed/v8processedText1.parquet...
Extracting Conclusions section...
Extracting Action plan and discussion section...
Extracting Discussion section...
Extracting Conclusion section...
Extracting CONCLUSION section...
Extracting Discussion: section...
Extracting Conclusion remarks ::: Introduction section...
Extracting 11. Conclusion section...
Extracting Concluding remarks section...
Extracting Conclusion. :::  section...
Extracting (iii) What comorbidities are associated with severe disease outcome and how do these affect viral pathogenesis? :::  section...
Extracting Conclusion and future prospects section...
Extracting Closing remarks section...
Extracting RESULTS AND DISCUSSION section...
Extracting Conclusion ::: Low-dose steroids and NSAIDs ::: Results section...
Extracting Conclusion ::: Mycophenolate ::: Results section...
Extracting Conclusion ::: IL-6 blockade ::: Results section...
Extracting Overall conclusion section...
Extracting Conclusion :

Extracting Conclusions and Resources section...
Extracting The relationship between CT imaging and clinical outcome section...
Extracting Conclusion and Discussion section...
Extracting Clinical and Laboratory Data and Outcome Assessment section...
Extracting IX. SUMMARY AND CONCLUSION section...
Extracting Test Result Interpretation section...
Extracting Primary Outcome section...
Extracting Assessment and outcome section...
Extracting Secondary Outcome section...
Extracting Clinical outcomes of pregnant women with COVID-19 section...
Extracting Case presentation and discussion section...
Extracting Summary and Discussion section...
Extracting Treatment and outcomes section...
Extracting Secondary outcome，median(IQR) section...
Extracting Primary and secondary outcomes in all patients section...
Extracting Primary and secondary outcomes in subgroups section...
Extracting Secondary outcome section...
Extracting Discussion and Conclusion section...
Extracting Pregnancy outcomes followin

Extracting Treatment and clinical outcome section...
Extracting Concluding remarks: from focusing on pathogens to understanding virus species section...
Extracting Final Concluding Remarks based on our COVID-19 Predictions in China section...
Extracting Conclusion B: section...
Extracting Conclusion A: section...
Extracting Conclusions on the reproductive number section...
Extracting d. Conclusions section...
Extracting IV -Discussion section...
Extracting Clinical Outcomes, Complications and Treatment section...
Extracting V. CONCLUSION AND DISCUSSION section...
Extracting Limitations and conclusion section...
Extracting Discussion 163 section...
Extracting RECOMMENDATIONS AND CONCLUSION section...
Extracting Discussion 230 section...
Extracting III. CONCLUSIONS section...
Extracting Discussion: Implications for COVID-19 section...
Extracting Conclusion: 217 section...
Extracting Patient follow-up and clinical outcomes section...
Extracting Follow-up and outcomes section...
Extracting

Extracting What is the duration of viral shedding? ::: DISCUSSION section...
Extracting What risk factors pre-dispose health care workers to infection or transmission? ::: DISCUSSION section...
Extracting What is the optimal diagnostic test? ::: DISCUSSION section...
Extracting What characteristics define a “case”? ::: DISCUSSION section...
Extracting What are the risk factors for infection (in patient population)? ::: DISCUSSION section...
Extracting What is the causative agent of disease? ::: DISCUSSION section...
Extracting Implications for practice ::: DISCUSSION section...
Extracting What are risk factors for death or severe illness? ::: DISCUSSION section...
Extracting Outcomes ::: Results section...
Extracting Conclusion and perspective section...
Extracting Risk Factors of Mental Health Outcomes ::: Results section...
Extracting Limitations ::: Discussion section...
Extracting Outcomes and Covariates ::: Methods section...
Extracting 4. Discussion section...
Extracting 4. Discu

Extracting Research in Context section...
Extracting Clinical Outcomes section...
Extracting Figure 3. Predicting positive testing outcome windows for diagnosed patients section...
Extracting Safety outcomes section...
Extracting Efficacy outcomes section...
Extracting VI. Conclusions VI. Conclusions section...
Extracting Treatment Situation and In-hospital Outcomes section...
Extracting DISCUSSION: section...
Extracting VII. CONCLUSIONS section...
Extracting FITTING RESULTS & INTERPRETATIONS section...
Extracting DISCUSSION & CONCLUSION section...
Extracting Primary outcome section...
Extracting Discussion 114 section...
Extracting Maternal outcomes section...
Extracting Outcomes 268 section...
Extracting DISCUSSIONS section...
Extracting Outcome scoring section...
Extracting Outcomes pool section...
Extracting Timeline and outcome of the disease progression 219 section...
Extracting Conclusion 353 section...
Extracting On the Interpretation of PIPE-Sites Predictions section...
Extrac

...

Extracting Clinical outcomes section...
Extracting Discussion. section...
Extracting Genetic associations with outcome variables section...
Extracting Concluding Thoughts section...
Extracting Study outcomes section...
Extracting Incidence and outcomes of patients with delayed-phase thrombocytopenia section...
Extracting Research in context Evidence before this study section...
Extracting CONCLUSIONS AND POTENTIAL RECOMMENDATIONS section...
Extracting Conclusions and Future Work section...
Extracting Conclusions and comments section...
Extracting Conclusion & Future Directions 132 section...
Extracting Outcomes: section...
Extracting CONCLUSION AND DISCUSSION section...
Extracting Conclusions 13 section...
Extracting Projections of SARS CoV-2 outcomes assuming remdesivir treatment during early and late symptomatic phases section...
Extracting Projections of SARS CoV-2 outcomes assuming extremely early remdesivir treatment during the pre-symptomatic phase section...
Extracting Figure 5.

Extracting Independent predictors for adverse clinical outcome section...
Extracting A. Interpretation of the public data section...
Extracting Interpretation and implications section...
Extracting Discussion 233 section...
Extracting Conclusions 266 section...
Extracting Outcome data section...
Extracting Outcomes and risk factors for longer hospitalization of COVID-19 section...
Extracting Discussion 193 section...
Extracting Estimation Results and Discussion section...
Extracting Discussion 224 section...
Extracting Endpoint outcome assessment section...
Extracting Conclusions and Discussion section...
Extracting Conclusions 345 section...
Extracting Conclusions and further directions section...
Extracting Treatment and clinical outcome section...
Extracting Concluding remarks: from focusing on pathogens to understanding virus species section...
Extracting Final Concluding Remarks based on our COVID-19 Predictions in China section...
Extracting Conclusion B: section...
Extracting Co

Extracting 4.1.2. Serological Testing ::: 4.1. Rapid Diagnostics ::: 4. Discussion section...
Extracting 4.3. Therapeutics ::: 4. Discussion section...
Extracting 4.2. Vaccines ::: 4. Discussion section...
Extracting 4.1.1. Detection of Genetic Material ::: 4.1. Rapid Diagnostics ::: 4. Discussion section...
Extracting Study records, data items and outcomes ::: Methods section...
Extracting Discussion and conclusion section...
Extracting Type of outcome measurements ::: Inclusion criteria ::: Eligibility criteria ::: Methods/design section...
Extracting What is the clinical presentation and spectrum of disease? ::: DISCUSSION section...
Extracting What treatments are effective (including the role of antivirals and steroids)? ::: DISCUSSION section...
Extracting What are the psychosocial consequences of infection with the virus? ::: DISCUSSION section...
Extracting Limitations ::: DISCUSSION section...
Extracting What is the duration of viral shedding? ::: DISCUSSION section...
Extracti

Extracting Clinical characteristics and outcomes of COVID-19 stratified by the number of comorbidities section...
Extracting DISCUSSION AND LIMITATION section...
Extracting Assessment of Exposure, Ascertainment of outcomes, Ascertainment of covariates section...
Extracting Results and discussions section...
Extracting Forecast and Discussion section...
Extracting Disease process and clinical outcomes section...
Extracting Main outcomes section...
Extracting Conclusions and Future directions section...
Extracting Outcome Measures and Definitions section...
Extracting Outcome of patients treated with CP as compared to a recent historic control groupAhistoric section...
Extracting Research in Context section...
Extracting Clinical Outcomes section...
Extracting Figure 3. Predicting positive testing outcome windows for diagnosed patients section...
Extracting Safety outcomes section...
Extracting Efficacy outcomes section...
Extracting VI. Conclusions VI. Conclusions section...
Extracting 

Extracting 3 Discussion and conclusion section...


In [308]:
"""
Add the extracted section headers/text for covid-19 papers to the covid19_title_abstract_dict.

"""

sect_header_uids = set([])

uid_headers_and_text_dict = {}

for sect_header, uid_texts in sect_dict.items():
    
    for uid, text in uid_texts.items():
        covid19_title_abstract_dict[uid][sect_header] = ' '.join(text)
for uid, sect_header_text in  covid19_title_abstract_dict.items():
    if uid not in uid_headers_and_text_dict:
        uid_headers_and_text_dict[uid] = {}
    uid_headers_and_text_dict[uid]['text'] = sect_header_text
    uid_section_headers = uid_section_headers_dict.get(uid, {'title', 'abstract'})
    uid_headers_and_text_dict[uid]['available_headers'] = list(uid_section_headers)

In [309]:
"""
Write dict to json

"""

with open('uids_section_header_and_texts_dict_200427.json', 'w') as f:
    json.dump(uid_headers_and_text_dict, f)

In [3]:
for uid, headers_text in uid_headers_and_text_dict.items():
    for header, text in headers_text['text'].items():
        print(uid, header, len(text))

NameError: name 'uid_headers_and_text_dict' is not defined

In [316]:

with open("covid_vt_contra_outcome_text_examples.txt", 'w') as f:
    for uid, header_text in uid_headers_and_text_dict.items():
        if 'Outcomes' in (header_text['available_headers']):
            f.write('%s\n' % uid)
            f.write('%s\n' % header_text['text']['Outcomes'])
            f.write("\n")
            
        for header in header_text['available_headers']:
            if 'outcome' in header:
                print(header)

(iii) What comorbidities are associated with severe disease outcome and how do these affect viral pathogenesis? ::: 
Adjusting for outcome delay in case fatality ratio estimates
Clinical course and outcomes ::: RESULTS
General clinical characteristics and outcomes of Chinese patients with COVID-19
The efficacy outcome ::: MSC treatment procedure and general patient information ::: RESULTS
The primary safety outcome ::: MSC treatment procedure and general patient information ::: RESULTS
Type of outcome measurements ::: Inclusion criteria ::: Eligibility criteria ::: Methods/design
Study records, data items and outcomes ::: Methods
Definition of Clinical outcomes ::: Materials and Method
Clinical outcomes ::: Results
Imaging, complications, and outcomes
Postoperative outcomes
Personal protective equipment (PPE) preparation and outcome
Overall clinical features and outcome
Complications, treatment and clinical outcomes during hospitalization and follow-up 296
The relationship between CT i

In [ ]:
merge